In [3]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import hstack

In [8]:
df=pd.read_csv('/Users/isaromobru/Desktop/FakeNews 10.57.24/proyecto_machine_learning_Fake_News/notebooks/processedstopword.csv')

In [9]:
le = LabelEncoder()
df['subject_encoded'] = le.fit_transform(df['subject'])


In [10]:
# Eliminar Na
df['text'] = df['text'].fillna("")
df['title'] = df['title'].fillna("")
df['subject'] = df['subject'].fillna("")

tfidf_title = TfidfVectorizer(max_features=5000)  # Limitar palabras para optimización
tfidf_text = TfidfVectorizer(max_features=10000)

X_title = tfidf_title.fit_transform(df['title'])
X_text = tfidf_text.fit_transform(df['text'])



In [11]:
# Convertir subject_encoded a formato de matriz dispersa
X_subject = np.array(df['subject_encoded']).reshape(-1,1)

# Concatenar todas las variables
X = hstack((X_subject, X_title, X_text))

# Definir la variable objetivo
y = df['label']

In [13]:



# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el pipeline que incluye el preprocesamiento y el ensamble de modelos
pipeline = Pipeline([
    
    ('ensemble', VotingClassifier(estimators=[
         ('naive_bayes', MultinomialNB()),
         ('svc', SVC(probability=True, random_state=42)),
         ('random_forest', RandomForestClassifier(random_state=42)),
         ('logistic_regression', LogisticRegression(solver='liblinear', random_state=42)),
         ('gradient_boosting', GradientBoostingClassifier(random_state=42))
    ], voting='soft'))
])

# Entrenar el pipeline
pipeline.fit(X_train, y_train)

# Realizar predicciones y evaluar el desempeño
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy del ensemble:", accuracy)


Accuracy del ensemble: 1.0
